In [39]:
##### Cleans Brazil capital stock data
# extracts from excel format and cleans

import os
import pandas as pd
import numpy as np


In [40]:
##### Load data

# Get the current working directory
cd = os.path.dirname(os.getcwd())

# Import data
BRA_codes = pd.read_csv(f"{cd}/Data/Correspondence_tables/BRA_municipalities.csv")

# Set save path
save_path = f"{cd}/Data/Clean/Capital_stock/BRA_capital_stock_SIDRA.csv"

In [41]:
##### Define function to clean IBGE spreadsheet

def clean_table(path):

    df = pd.read_excel(path, header=None, dtype=str)

    df = df.dropna(how="all")

    # set correct header row
    header_row = df[df.apply(lambda x: x.astype(str).str.contains("Município", na=False).any(), axis=1)].index[0]
    df.columns = df.iloc[header_row].fillna(method="ffill").astype(str)

    # crop unneccesary data
    df = df.iloc[header_row + 1:].reset_index(drop=True)
    df = df[[
        "Município",
        "Ano x Tipologia x Tratores, implementos e máquinas existentes no estabelecimento agropecuário"
    ]].copy()

    df = df.dropna(subset=["Município"]).reset_index(drop=True)

    # extract municipality name and state code
    df[["municipality_name", "state"]] = df["Município"].str.extract(r'^(.*)\s+\(([A-Z]{2})\)\s*$')

    df = df.rename(columns={
        "Ano x Tipologia x Tratores, implementos e máquinas existentes no estabelecimento agropecuário": "2017"
    })
    df["2017"] = pd.to_numeric(df["2017"], errors="coerce")

    df = df.dropna()

    return df[["municipality_name", "state", "2017"]]

In [42]:
state_dict = {
    "RO": "Rondônia",
    "AC": "Acre",
    "AM": "Amazonas",
    "RR": "Roraima",
    "PA": "Pará",
    "AP": "Amapá",
    "TO": "Tocantins",
    "MA": "Maranhão",
    "PI": "Piauí",
    "CE": "Ceará",
    "RN": "Rio Grande do Norte",
    "PB": "Paraíba",
    "PE": "Pernambuco",
    "AL": "Alagoas",
    "SE": "Sergipe",
    "BA": "Bahia",
    "MG": "Minas Gerais",
    "ES": "Espírito Santo",
    "RJ": "Rio de Janeiro",
    "SP": "São Paulo",
    "PR": "Paraná",
    "SC": "Santa Catarina",
    "RS": "Rio Grande do Sul",
    "MS": "Mato Grosso do Sul",
    "MT": "Mato Grosso",
    "GO": "Goiás",
    "DF": "Distrito Federal"
}

In [43]:
##### Clean 

df = clean_table(f"{cd}/Data/Raw/Brazil/table6874.xlsx")

df["state_name"] = df["state"].map(state_dict)

# create MATCH column
df['MATCH'] = df['state_name'] + df['municipality_name']
BRA_codes['MATCH'] = BRA_codes['STATE_name'] + BRA_codes['IBGE_name']

# merge to get admin codes
capital = df.merge(BRA_codes, on='MATCH', how='outer')

# add units
capital['Units'] = 'Ag capital stock - count of tractors, machinery, and equipment'

# reorder columns
columns_to_keep = ['MUNIP_CODE', 'Units', '2017']
capital = capital[columns_to_keep]

/var/folders/48/ky2jtbmj31bfj15cr5gq480w0000gn/T/ipykernel_1685/3279417715.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.columns = df.iloc[header_row].fillna(method="ffill").astype(str)


In [44]:
##### Save cleaned data
capital.to_csv(save_path, index=False)